In [3]:
import numpy as np
import pandas as pd

In [74]:
df=pd.read_csv("~/Downloads/NFL_data_cleaned.csv")
orig=pd.read_csv("~/Downloads/archive/NFL Play by Play 2009-2018 (v5).csv")
#orig is the original dataset we had, importing because one column was mistakenly deleted in the cleaned version so I will add it back in

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
pd.unique(df['play_type'])
len(orig)==len(df)
score_margin=orig['score_differential']
df['score_margin']=score_margin
df
#attaching the score_differential column to df

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,play_type,yards_gained,score_margin
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,9/10/09,...,1,NaN,0.0,15:00,TEN 30,0,0,kickoff,0.0,NaN
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,9/10/09,...,1,1.0,0.0,14:53,PIT 42,10,5,pass,5.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,9/10/09,...,1,2.0,0.0,14:16,PIT 47,5,2,run,-3.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,9/10/09,...,1,3.0,0.0,13:35,PIT 44,8,2,pass,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,9/10/09,...,1,4.0,0.0,13:27,PIT 44,8,2,punt,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449366,4059,2018121700,CAR,NO,CAR,home,NO,CAR,66.0,12/17/18,...,4,2.0,0.0,1:03:00,CAR 34,10,19,pass,0.0,-5.0
449367,4081,2018121700,CAR,NO,CAR,home,NO,CAR,66.0,12/17/18,...,4,3.0,0.0,0:58:00,CAR 34,10,19,pass,5.0,-5.0
449368,4106,2018121700,CAR,NO,CAR,home,NO,CAR,61.0,12/17/18,...,4,4.0,0.0,0:38:00,CAR 39,5,19,pass,0.0,-5.0
449369,4128,2018121700,CAR,NO,NO,away,CAR,CAR,39.0,12/17/18,...,4,1.0,0.0,0:35:00,CAR 39,10,-1,qb_kneel,-1.0,5.0


In [77]:
df.replace(to_replace='punt', value='kick', inplace=True)
df.replace(to_replace='field_goal', value='kick', inplace=True)
pd.unique(df['play_type'])
#condensing "punt" and "field_goal" plays into one category

array(['kickoff', 'pass', 'run', 'kick', 'no_play', nan, 'extra_point',
       'qb_kneel', 'qb_spike'], dtype=object)

In [78]:
df=df.drop(df.index[df['play_type']=='kickoff'])
df=df.drop(df.index[df['play_type']=='no_play'])
df=df.drop(df.index[df['play_type']=='qb_kneel'])
df=df.drop(df.index[df['play_type']=='extra_point'])
df=df.drop(df.index[df['play_type']=='qb_spike'])
#removing irrelevant play types
#no_play means there was a penalty or a stoppage and there wasn't actually a play (irrelevant to our model)
#kickoff is obvious when a game is going on, the team makes it obvious they are going to kick off
#qb_kneel and qb_spike are not really "plays", they are done when a team is trying to either waste time or conserve time
#and they are not relevant to our model because it is also obvious when they are going to spike or kneel
#extra_point was removed because this only comes after a touchdown and it is also obvious when they are going to attempt an extra point
df=df.dropna(subset=['play_type'])
#There were some NA play_types due to quarters ending, so we want to remove those because those are not plays

In [79]:
df=df.drop(columns=['home_team', 'play_id', 'game_id', 'away_team', 'posteam_type', 'defteam', 'side_of_field', 'game_date', 'sp', 'time', 'yrdln', 'ydsnet', 'yards_gained', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive'])
#extra columns

In [80]:
pd.unique(df['play_type'])
pd.unique(df['score_margin'])
df
#examine df and unique values of these columns to make sure cleaning was done correctly and there are no NA score 

,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
1,PIT,58.0,893.0,1,1.0,0.0,10,pass,0.0
2,PIT,53.0,856.0,1,2.0,0.0,5,run,0.0
3,PIT,56.0,815.0,1,3.0,0.0,8,pass,0.0
4,PIT,56.0,807.0,1,4.0,0.0,8,kick,0.0
5,TEN,98.0,796.0,1,1.0,0.0,10,run,0.0
...,...,...,...,...,...,...,...,...,...
449363,CAR,71.0,82.0,4,2.0,0.0,1,pass,-5.0
449364,CAR,71.0,77.0,4,3.0,0.0,1,pass,-5.0
449366,CAR,66.0,63.0,4,2.0,0.0,10,pass,-5.0
449367,CAR,66.0,58.0,4,3.0,0.0,10,pass,-5.0


In [82]:
LVc=len(df[df['posteam'] == 'OAK'])
LARc=len(df[df['posteam'] == 'LA'])+len(df[df['posteam'] == 'STL'])
LACr=len(df[df['posteam'] == 'SD'])+len(df[df['posteam'] == 'LAC'])
JAXc=len(df[df['posteam'] == 'JAX'])+len(df[df['posteam'] == 'JAC'])

In [86]:
#Above I stored the values for how many instances of each new abbreviation there should be, and I will
#compare later to make sure I did the cleaning correctly
#we need to change the abbreviations for teams who either relocated or had their abbreviation change between 2009 and now
#Jacksonville had their abbreviation change from JAC to JAX
#St. Louis relocated and changed from STL to LA to LAR, so I am going to change instances of STL and LA both to LAR
#San Diego relocated and had their abbreviation change from SD to LAC
#Oakland relocated and had their abbreviation change from OAK to LV
pd.unique(df['posteam'])
df.replace(to_replace='JAC', value='JAX', inplace=True)
df.replace(to_replace='STL', value='LAR', inplace=True)
df.replace(to_replace='LA', value='LAR', inplace=True)
df.replace(to_replace='OAK', value='LV', inplace=True)
df.replace(to_replace='SD', value='LAC', inplace=True)
pd.unique(df['posteam'])

array(['PIT', 'TEN', 'CLE', 'MIN', 'NO', 'DET', 'DAL', 'TB', 'HOU', 'NYJ',
       'IND', 'JAX', 'DEN', 'CIN', 'CAR', 'PHI', 'KC', 'BAL', 'ATL',
       'MIA', 'ARI', 'SF', 'SEA', 'LAR', 'NYG', 'WAS', 'GB', 'CHI', 'NE',
       'BUF', 'LV', 'LAC'], dtype=object)

In [87]:
len(pd.unique(df['posteam']))==32

True

In [92]:
LVc==len(df[df['posteam'] == 'LV'])
LARc==len(df[df['posteam'] == 'LAR'])
LACr==len(df[df['posteam'] == 'LAC'])
JAXc==len(df[df['posteam'] == 'JAX'])

,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
1,PIT,58.0,893.0,1,1.0,0.0,10,pass,0.0
2,PIT,53.0,856.0,1,2.0,0.0,5,run,0.0
3,PIT,56.0,815.0,1,3.0,0.0,8,pass,0.0
4,PIT,56.0,807.0,1,4.0,0.0,8,kick,0.0
5,TEN,98.0,796.0,1,1.0,0.0,10,run,0.0
...,...,...,...,...,...,...,...,...,...
449363,CAR,71.0,82.0,4,2.0,0.0,1,pass,-5.0
449364,CAR,71.0,77.0,4,3.0,0.0,1,pass,-5.0
449366,CAR,66.0,63.0,4,2.0,0.0,10,pass,-5.0
449367,CAR,66.0,58.0,4,3.0,0.0,10,pass,-5.0


In [93]:
df.to_csv("~/Downloads/NFL_data_super_cleaned.csv", index=False)